In [1]:
import tkinter as tk
import threading
import time
from PIL import Image, ImageTk
import numpy as np

root = tk.Tk()

In [2]:
def center_window(root, width, height):
    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()
    x = (screen_width - width) // 2
    y = (screen_height - height) // 2
    root.geometry(f'{width}x{height}+{x}+{y}')

root.title("8_Queen")
center_window(root, 960, 640)
root.configure(bg='#161512')
root.resizable(False, False)

step_unstop_var = tk.IntVar()

''

In [3]:
class Point_value:
    def _init_(self, x, y, value):
        self.x = x
        self.y = y
        self.value = value

Showing_board = []
sum = 0
#Running_board = []

Q_img = Image.open("D:\workplace\LightQueen.png")
Q_img = Q_img.convert("RGBA")
Q_img = Q_img.resize((56, 56), Image.LANCZOS)
Q_img_show = ImageTk.PhotoImage(Q_img)

In [4]:
BOARD_SIZE = 8
chess_board = [[0 for _ in range(BOARD_SIZE)] for _ in range(BOARD_SIZE)]
solutions = []
saver_list = [0 for _ in range(BOARD_SIZE)]
img_list = []

def is_safe(board, row, col):
    for i in range(col):
        if board[row][i] == 1:
            return False
    for i, j in zip(range(row, -1, -1), range(col, -1, -1)):
        if board[i][j] == 1:
            return False
    for i, j in zip(range(row, 8, 1), range(col, -1, -1)):
        if board[i][j] == 1:
            return False
    return True

def solve_queen(board, col, solutions):
    global img_list
    if col >= 8:
        global sum
        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                if board[i][j] == 1:
                    saver_list[i] = j
        solutions.append(saver_list)
        sum = sum + 1
        if step_unstop_var == 0:
            event.wait()
        return
    for i in range(8):
        if is_safe(board, i, col):
            board[i][col] = 1
            color = '#f0d9b5' if (i + col) % 2 == 0 else '#b58863'
            label = tk.Label(root, image=Q_img_show, bg = color)
            label.place(x=Showing_board[i][col].x, y=Showing_board[i][col].y)
            root.update_idletasks()
            solve_queen(board, col + 1, solutions)
            board[i][col] = 0
            label.destroy()
            label = tk.Canvas(root, width=60, height=60, bg=color, highlightthickness=0)
            label.place(x=Showing_board[i][col].x, y=Showing_board[i][col].y)
            root.update_idletasks()
            label.destroy()

In [5]:
def finished_signal():
    signal_window = tk.Toplevel(root)
    signal_window.title("已完成")
    center_window(signal_window, 160, 50)
    signal_window.resizable(False, False)
    label = tk.Label(signal_window, text="已完成：共" + str(sum) + "个解", 
             bg='#161512',  fg='#cacaca',  font=("黑体", 12, 'bold'), 
             width=20,  height=5)
    signal_window.grid_rowconfigure(0, weight=1)
    signal_window.grid_columnconfigure(0, weight=1)
    label.grid(row=0, column=0, sticky="nsew")

def solving():
    solve_queen(chess_board, 0, solutions)
    finished_signal()
    buttons[1].config(state=tk.DISABLED)

In [6]:
def run_order():
    buttons[0].config(state=tk.DISABLED)
    buttons[1].config(state=tk.NORMAL)
    buttons[4].config(state=tk.NORMAL)
    checkbuttons[0].config(state=tk.DISABLED)
    checkbuttons[1].config(state=tk.DISABLED)
    solving()

def pause_order():
    buttons[1].config(state=tk.DISABLED)
    buttons[2].config(state=tk.NORMAL)
    buttons[3].config(state=tk.NORMAL)

def step_order():
    print("click")

def conti_order():
    buttons[1].config(state=tk.NORMAL)
    buttons[2].config(state=tk.DISABLED)
    buttons[3].config(state=tk.DISABLED)

def stop_order():
    chess_board.clear()
    Showing_board.clear()
    sum = 0
    solutions.clear()
    saver_list.clear()
    buttons[0].config(state=tk.NORMAL)
    buttons[1].config(state=tk.DISABLED)
    buttons[2].config(state=tk.DISABLED)
    buttons[3].config(state=tk.DISABLED)
    buttons[4].config(state=tk.DISABLED)
    checkbuttons[0].config(state=tk.NORMAL)
    checkbuttons[1].config(state=tk.NORMAL)

In [7]:
def create_chessboard(parent, x, y, side, square, saver_board):
    border_canvas = tk.Canvas(parent, width=(side + 8), height=(side + 8))
    border_canvas.place(x=x - 6, y=y - 6)
    border_color = "#2a2724"
    border_thickness = 20
    border_canvas.create_rectangle(0, 0, side, side,
                                   outline=border_color, width=border_thickness)
    square_size = square
    for row in range(8):
        row_point = []
        for col in range(8):
            color = '#f0d9b5' if (row + col) % 2 == 0 else '#b58863'
            square = tk.Canvas(parent, width=square_size, height=square_size, bg=color, highlightthickness=0)
            square.place(x=x + col * square_size, y=y + row * square_size)
            row_point.append(Point_value())
            row_point[col].x = x + col * square_size
            row_point[col].y = y + row * square_size
            row_point[col].value = 0
        saver_board.append(row_point)

In [8]:
create_chessboard(root, 50, 50, 480, 60, Showing_board)
#create_chessboard(root, 650, 350, 176, 22, Running_board)

buttons = [tk.Button(root, text="运行", command=run_order), 
           tk.Button(root, text="暂停", command=pause_order, state=tk.DISABLED), 
           tk.Button(root, text="步进", command=step_order, state=tk.DISABLED), 
           tk.Button(root, text="继续", command=conti_order, state=tk.DISABLED), 
           tk.Button(root, text="清空", command=stop_order, state=tk.DISABLED), ]
for button in buttons:
    button.configure(compound='left', bg="#51504d", fg="#cacaca", font=("黑体", 12, 'bold'), height=1, width=8)
buttons[0].place(x=570, y=240)
buttons[1].place(x=670, y=240)
buttons[2].place(x=770, y=240)
buttons[3].place(x=670, y=280)
buttons[4].place(x=770, y=280)

checkbuttons = [tk.Checkbutton(root, text="", onvalue=1, offvalue=0), 
                tk.Checkbutton(root, text="", onvalue=2, offvalue=0)]
for checkbutton in checkbuttons:
    checkbutton.configure(variable=step_unstop_var, bg="#161512", fg="#161512",  activebackground='#676664')
checkbuttons[0].place(x=650, y=170)
label_text = "步进运行"
label = tk.Label(root, text=label_text, fg="#cacaca", bg="#161512", font=("黑体", 12, 'bold'))
label.place(x=680, y=172)
checkbuttons[1].place(x=650, y=200)
label_text = "连续求解"
label = tk.Label(root, text=label_text, fg="#cacaca", bg="#161512", font=("黑体", 12, 'bold'))
label.place(x=680, y=202)
def handle_selection(_):
    value = step_unstop_var.get()
    if value == 1:
        checkbuttons[0].deselect()
    elif value == 2:
        checkbuttons[1].deselect()
checkbuttons[0].configure(command=handle_selection(_))
checkbuttons[1].configure(command=handle_selection(_))

root.mainloop()